In [6]:
import os
from bs4 import BeautifulSoup
import requests
import time
from datetime import datetime
import json
from zipfile import ZipFile
import pandas as pd
import shutil
import zipfile
import random

### Define PATH constant

In [7]:
BASE_PATH = ""
HTML_PATH = BASE_PATH + "/html"
USER_PATH = BASE_PATH + "/users"

# 1. Get Clubs IDs

In [136]:
def get_number(string):
    return int(string.strip().replace(",", ""))


clubs_id = set()
possibles_users = 0
page = 1

while True:
    print(f"\r{page}", end="")

    time.sleep(3)  # Wait 3 seconds per page
    data = requests.get(f"https://myanimelist.net/clubs.php?p={page}")
    soup = BeautifulSoup(data.text, "html.parser")
    rows = soup.find_all("tr", {"class": "table-data"})
    for row in rows:
        members = get_number(row.find("td", {"class": "ac"}).text)
        club_id = get_number(
            row.find("a", {"class": "fw-b"}).get("href").split("=")[-1]
        )
        if (
            club_id not in clubs_id and members > 30
        ):  # Only save groups with more than 30 members
            possibles_users += members
            clubs_id.add(club_id)

    page += 1
    if possibles_users > 1000000:  # Threshold to stop
    # if possibles_users > 10:  # Threshold to stop
        break

with open(f"{BASE_PATH}/clubs.txt", "w") as file:
    for club in clubs_id:
        file.write(f"{club}\n")

1

# 2. Get usernames in every clubs

In [137]:
if not os.path.exists(f"{BASE_PATH}/users_list.txt"):
    with open(f"{BASE_PATH}/users_list.txt", "w", encoding="UTF-8") as file:
        pass
    
if not os.path.exists(f"{BASE_PATH}/_revised_clubs.txt"):
    with open(f"{BASE_PATH}/_revised_clubs.txt", "w", encoding="UTF-8") as file:
        pass

In [138]:
with open(f"{BASE_PATH}/clubs.txt") as file:
    clubs_id = [x.strip() for x in file.readlines()]

with open(f"{BASE_PATH}/users_list.txt", encoding="UTF-8") as file:
    users = set([x.strip() for x in file.readlines()])

with open(f"{BASE_PATH}/_revised_clubs.txt", encoding="UTF-8") as file:
    revised_clubs = set([int(x.strip()) for x in file.readlines()])

len(users), len(revised_clubs), len(clubs_id)

(1282, 0, 41)

In [139]:
%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [140]:
%%notify -m "get username finish"

for i, club_id in enumerate(clubs_id):
    if club_id in revised_clubs:
        continue

    # page = 1
    # while True:
    print(f"\r{i+1}/{len(clubs_id)} --> {str(page).zfill(2)}", end="")
    # link = f"https://api.jikan.moe/v3/club/{club_id}/members/{page}"
    link = f"https://api.jikan.moe/v4/clubs/{club_id}/members"

    try:
        time.sleep(4.2)
        data = requests.get(link)
    except KeyboardInterrupt:
        raise KeyboardInterrupt()
    except:  # Other exception wait 2 min and try again
        time.sleep(120)
        continue

    if data.status_code != 200:
        break

    with open(f"{BASE_PATH}/users_list.txt", "a", encoding="UTF-8") as file:
        for user in map(lambda x: x["username"], json.loads(data.text)["data"]):
            if user not in users and user != "":
                file.write(f"{user}\n")
                users.add(user)
        # page += 1

    revised_clubs.add(club_id)
    with open(f"{BASE_PATH}/_revised_clubs.txt", "a", encoding="UTF-8") as file:
        file.write(f"{club_id}\n")

41/41 --> 02

<IPython.core.display.Javascript object>

In [141]:
with open(f"{BASE_PATH}/users_list.txt", encoding="UTF-8") as file:
    users = list(set([x.strip() for x in file.readlines()]))[1:]
    random.shuffle(users)

with open(f"{BASE_PATH}/users.csv", "w", encoding="UTF-8") as file:
    file.write("user_id,username\n")
    for i, user in enumerate(users):
        file.write(f"{i},{user}\n")

# 3. Get animelist per user

In [142]:
with open(f"{BASE_PATH}/users.csv", "r", encoding="UTF-8") as file:
    file.readline()
    users = [x.strip().split(",") for x in file.readlines()]
    users = [(int(x[0]), x[1]) for x in users]

last_revised_users = -1
if os.path.exists(f"{BASE_PATH}/_last_revised_users.txt"):
    with open(f"{BASE_PATH}/_last_revised_users.txt", "r", encoding="UTF-8") as file:
        last_revised_users = int(file.readline())

len(users), last_revised_users

(1281, -1)

In [143]:
%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [145]:
%%notify -m "animelist finish"
for i, (user_id, username) in enumerate(users):
    if user_id <= last_revised_users:
        continue

    now = datetime.now()
    print(f'\r{str(now).split(".")[0]} --> {i+1}/{len(users)}', end="")
    page = 1
    all_animes = []
    offset = 0
    while True:
    # link = f"https://api.jikan.moe/v4/user/{username}/animelist/all?page={page}"
        link = f"https://myanimelist.net/animelist/{username}/load.json?offset={offset}"
        try:
            time.sleep(4.2)
            data = requests.get(link, timeout=15)
        except KeyboardInterrupt:
            raise KeyboardInterrupt()
        except:  # Other exception wait 2 min and try again
            time.sleep(120)
            continue


        # if data.status_code != 200:
        #     break

        data = json.loads(data.text)

        for anime in data:
            try:
                all_animes.append((anime["anime_id"], anime["score"], anime["status"], anime["num_watched_episodes"]))
            except:
                print(username)

        offset += 300
        if len(data) < 300:
            break

    if len(all_animes) != 0:
        with open(f"{USER_PATH}/{user_id}.csv", "w") as f1:
            f1.write(f"anime_id,score,watching_status,watched_episodes\n")
            for anime_id, anime_score, watching_status, watched_episodes in all_animes:
                f1.write(
                    f"{anime_id},{anime_score},{watching_status},{watched_episodes}\n"
                )

    revised_users = user_id
    with open(f"{BASE_PATH}/_last_revised_users.txt", "w", encoding="UTF-8") as file:
        file.write(f"{user_id}\n")

2023-01-05 11:53:39 --> 27/1281Cutebunny6
2023-01-05 11:55:55 --> 36/1281Amenophyckxs
2023-01-05 11:56:36 --> 40/12812ndchannel
2023-01-05 11:57:50 --> 43/1281andreea_
2023-01-05 11:57:55 --> 44/1281-Ryoto-
2023-01-05 11:59:40 --> 56/1281Ahxxn
2023-01-05 12:00:07 --> 59/1281BakuKafuu
2023-01-05 12:05:11 --> 77/1281gulabi
2023-01-05 12:05:39 --> 80/1281Amulet_Heart
2023-01-05 12:07:28 --> 89/1281--Tapioca--
2023-01-05 12:11:11 --> 110/1281aferrari
2023-01-05 12:16:28 --> 136/1281-Karoshi-
2023-01-05 12:27:06 --> 174/1281-Based-
2023-01-05 12:27:24 --> 177/1281Alwarja
2023-01-05 12:30:56 --> 188/1281Daianmoi
2023-01-05 12:32:08 --> 195/1281-HippySnob-
2023-01-05 12:33:39 --> 202/1281falcon202020
2023-01-05 12:36:43 --> 215/1281-MisT-
2023-01-05 12:38:56 --> 226/1281-Bass
2023-01-05 12:41:02 --> 236/1281Elfinstones
2023-01-05 12:50:55 --> 281/1281Masterbrawler
2023-01-05 13:03:27 --> 318/1281Apis_Mellifera_
2023-01-05 13:03:42 --> 321/1281Honrychamp
2023-01-05 13:12:22 --> 354/1281-aejay-

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

# 4. Download Anime HTML

In [8]:
unique_anime = set()
folder = os.listdir(USER_PATH)
for i, user_file in enumerate(folder):
    if ".csv" not in user_file:
        continue

    print(f"\r{i + 1}/{len(folder)}", end="")
    with open(f"{USER_PATH}/{user_file}", "r") as file:
        file.readline()
        for line in file:
            anime = line.strip().split(",")[0]
            unique_anime.add(anime)

print("         ")
print(len(unique_anime))

577/577         
21436


In [9]:
MAX = 7  # MAX SECOND TO WAIT PER REQUEST
MIN = 4  # MIN SECONDS TO WAIT PER REQUEST


def sleep():
    time_to_sleep = random.random() * (MAX - MIN) + MIN
    time.sleep(time_to_sleep)


def get_link_by_text(soup, anime_id, text):
    links = list(filter(lambda x: anime_id in x["href"], soup.find_all("a", text=text)))
    return links[0]["href"]


def save(path, data):
    with open(path, "w", encoding="UTF-8") as file:
        file.write(data)


def save_link(link, anime_id, name):
    sleep()
    path = f"{HTML_PATH}/{anime_id}/{name}.html"
    data = requests.get(link)
    soup = BeautifulSoup(data.text, "html.parser")
    soup.script.decompose()
    save(path, soup.prettify())
    return soup


def save_reviews(link, anime_id):
    page = 1
    while True:
        sleep()
        actual_link = f"{link}?p={page}"
        data = requests.get(actual_link)
        soup = BeautifulSoup(data.text, "html.parser")
        reviews = soup.find_all("a", text="Overall Rating")
        if len(reviews) == 0:
            break

        path = f"{HTML_PATH}/{anime_id}/reviews_{page}.html"
        soup.script.decompose()
        save(path, soup.prettify())
        page += 1


def scrap_anime(anime_id):
    path = f"{HTML_PATH}/{anime_id}"
    os.makedirs(path, exist_ok=True)
    sleep()
    data = requests.get(f"https://myanimelist.net/anime/{anime_id}")

    anime_info = data.text
    soup = BeautifulSoup(anime_info, "html.parser")
    soup.script.decompose()
    save(f"{HTML_PATH}/{anime_id}/details.html", soup.prettify())

    link_review = get_link_by_text(soup, anime_id, "Reviews")
    link_recomendations = get_link_by_text(soup, anime_id, "Recommendations")
    link_stats = get_link_by_text(soup, anime_id, "Stats")
    link_staff = get_link_by_text(soup, anime_id, "Characters & Staff")
    link_pictures = get_link_by_text(soup, anime_id, "Pictures")

    save_link(link_pictures, anime_id, "pictures")
    save_link(link_staff, anime_id, "staff")
    save_link(link_stats, anime_id, "stats")
    save_link(link_recomendations, anime_id, "recomendations")
    save_reviews(link_review, anime_id)


def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(
                os.path.join(root, file),
                os.path.relpath(os.path.join(root, file), path),
            )


In [10]:
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [11]:
%%notify -m "Anime scrapping finish"

for i, anime_id in enumerate(unique_anime):
    if os.path.isfile(f"{HTML_PATH}/{anime_id}.zip"):
        continue

    print(f"\r{i+1}/{len(unique_anime)}", end="")

    try:
        scrap_anime(anime_id)
    except KeyboardInterrupt:
        break
    except:  # Other exception wait 2 min and try again
        time.sleep(120)
        continue

    path = f"{HTML_PATH}/{anime_id}"
    zipf = zipfile.ZipFile(f"{path}.zip", "w", zipfile.ZIP_DEFLATED)
    zipdir(path, zipf)
    zipf.close()

    shutil.rmtree(path)

3/21436

<IPython.core.display.Javascript object>

# 5. Scrapping Anime info from HTML

In [27]:
def extract_zip(input_zip):
    input_zip = ZipFile(input_zip)
    return {name: input_zip.read(name) for name in input_zip.namelist()}

KEYS = ['MAL_ID', 'Name', 'Score', 'Genders', 'English name', 'Japanese name', 'Type', 'Episodes',
        'Aired', 'Premiered', 'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating',
        'Ranked', 'Popularity', 'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
        'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6', 'Score-5', 'Score-4',
        'Score-3', 'Score-2', 'Score-1']

In [28]:
def get_name(info):
    try:
        return info.find("h1", {"class": "title-name h1_bold_none"}).text.strip()
    except: 
        return ""

def get_english_name(info):
    try:
        span = info.findAll("span", {"class": "dark_text"})
        return span.parent.text.strip()
    except:
        return ""

def get_table(a_soup):
    try:
        return a_soup.find("div", {"class": "po-r js-statistics-info di-ib"})
    except:
        return ""

def get_score(stats):
    try:
        score = stats.find("span", {"itemprop": "ratingValue"})
        if score is None:
            return "Unknown"
        return score.text.strip()
    except:
        return ""

def get_gender(sum_info):
    try:
        text = ", ".join(
            [x.text.strip() for x in sum_info.findAll("span", {"itemprop": "genre"})]
        )
        return text
    except:
        return ""

def get_description(sum_info):
    try:
        return sum_info.find("td", {"class": "borderClass", "width": "225"})
    except:
        return ""

def get_all_stats(soup):
    try:
        return soup.find("div", {"id": "horiznav_nav"}).parent.findAll(
            "div", {"class": "spaceit_pad"}
        )
    except:
        return ""

def get_info_anime(anime_id):
    data = extract_zip(f"data/html/{anime_id}.zip")
    anime_info = data["stats.html"].decode()
    soup = BeautifulSoup(anime_info, "html.parser")

    stats = get_table(soup)
    description = get_description(soup)
    anime_info = {key: "Unknown" for key in KEYS}

    anime_info["MAL_ID"] = anime_id
    anime_info["Name"] = get_name(soup)
    anime_info["Score"] = get_score(stats)
    anime_info["Genders"] = get_gender(description)

    for d in description.findAll("span", {"class": "dark_text"}):
        information = [x.strip().replace(" ", " ") for x in d.parent.text.split(":")]
        category, value = information[0], ":".join(information[1:])
        value.replace("\t", "")

        if category in ["Broadcast", "Synonyms", "Genres", "Score", "Status"]:
            continue

        if category in ["Ranked"]:
            value = value.split("\n")[0]
        if category in ["Producers", "Licensors", "Studios"]:
            value = ", ".join([x.strip() for x in value.split(",")])
        if category in ["Ranked", "Popularity"]:
            value = value.replace("#", "")
        if category in ["Members", "Favorites"]:
            value = value.replace(",", "")
        if category in ["English", "Japanese"]:
            category += " name"

        anime_info[category] = value

    # Stats (Watching, Completed, On-Hold, Dropped, Plan to Watch)
    for d in get_all_stats(soup)[:5]:
        category, value = [x.strip().replace(" ", " ") for x in d.text.split(":")]
        value = value.replace(",", "")
        anime_info[category] = value

    # Stast votes per score
    for d in get_all_stats(soup)[6:]:
        score = d.parent.parent.find("td", {"class": "score-label"}).text.strip()
        value = [x.strip().replace(" ", " ") for x in d.text.split("%")][1].strip(
            "(votes)"
        )
        label = f"Score-{score}"
        anime_info[label] = value.strip()

    for key, value in anime_info.items():
        if str(value) in ["?", "None found, add some", "None", "N/A", "Not available"]:
            anime_info[key] = "Unknown"
    return anime_info

In [29]:
get_info_anime(49827)

{'MAL_ID': 49827,
 'Name': 'Kidou Senshi Gundam: Cucuruz Doan no Shima',
 'Score': '',
 'Genders': 'Action, Mecha',
 'English name': "Mobile Suit Gundam:Cucuruz Doan's Island",
 'Japanese name': '機動戦士ガンダム ククルス・ドアンの島',
 'Type': 'Movie',
 'Episodes': '1',
 'Aired': 'Jun 3, 2022',
 'Premiered': 'Unknown',
 'Producers': 'Bandai Namco Filmworks',
 'Licensors': 'Unknown',
 'Studios': 'Sunrise',
 'Source': 'Other',
 'Duration': '1 hr. 48 min.',
 'Rating': 'PG-13 - Teens 13 or older',
 'Ranked': '2308',
 'Popularity': '6936',
 'Members': '6547',
 'Favorites': '23',
 'Watching': '202',
 'Completed': '2775',
 'On-Hold': '68',
 'Dropped': '52',
 'Plan to Watch': '3450',
 'Score-10': '104',
 'Score-9': '261',
 'Score-8': '752',
 'Score-7': '729',
 'Score-6': '275',
 'Score-5': '93',
 'Score-4': '23',
 'Score-3': '10',
 'Score-2': '8',
 'Score-1': '9',
 'Genre': 'Action\n            \n\n             Action',
 'Theme': 'Mecha\n            \n\n             Mecha'}

In [30]:
# Generate anime.tsvdf.to_csv(f"{BASE_PATH}/anime.tsv", index=False, sep="\t", encoding="UTF-8")

anime_revised = set()
exist_file = os.path.exists(f"{BASE_PATH}/anime.tsv")
actual_data = pd.DataFrame()
if exist_file:
    # If the file exist, include new data.
    actual_data = pd.read_csv(f"{BASE_PATH}/anime.tsv", sep="\t")
    anime_revised = list(actual_data.MAL_ID.unique())

actual_data.head()
total_data = []
zips = os.listdir(HTML_PATH)
for i, anime in enumerate(zips):
    if not ".zip" in anime:
        continue

    anime_id = int(anime.strip(".zip"))

    if int(anime_id) in anime_revised:
        continue

    print(f"\r{i+1}/{len(zips)} ({anime_id})", end="")

    anime_id = anime.strip(".zip")
    total_data.append(get_info_anime(anime_id))

if len(total_data):
    df = pd.DataFrame.from_dict(total_data)
    df["MAL_ID"] = pd.to_numeric(df["MAL_ID"])
    df = df.sort_values(by="MAL_ID").reset_index(drop=True)

    if exist_file:
        df = (
            pd.concat([actual_data, df]).sort_values(by="MAL_ID").reset_index(drop=True)
        )

else:
    df = actual_data

pd.set_option("display.max_columns", None)
df.head()

5/5 (28891)

,MAL_ID,Name,Score,Genders,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Genre,Theme,German,Spanish,French,Themes,Demographic
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity),28,39,1251960,61971,105808,718161,71513,26678,329800,229170,182126,131625,62330,20688,8904,3184,1357,741,1580,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr. 55 min.,R - 17+ (violence & profanity),159,518,273145,1174,4143,208333,1935,770,57964,30043,49201,49505,22632,5805,1877,577,221,109,379,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,28891,Haikyuu!! Second Season,,"Sports, School, Team Sports, Shounen",Haikyu!! 2nd Season,ハイキュー!! セカンドシーズン,TV,25,"Oct 4, 2015 to Mar 27, 2016",Fall 2015,"Dentsu, Mainichi Broadcasting System, Movic, S...",Sentai Filmworks,Production I.G,Manga,24 min. per ep.,PG-13 - Teens 13 or older,70,84,1314392,15339,57549,1133329,16040,8312,99162,214776,286974,248547,89350,19823,6368,1580,604,353,1368,Sports\n \n\n Sports,NaN,Haikyu!! Staffel 2,Haikyu!! Los Ases del Vóley Temporada 2,Haikyu!! Saison 2,School\n \n\n School\n ...,Shounen\n \n\n Shounen
3,32796,Shenmi Shijie Lixian Ji,,"Adventure, Comedy, Fantasy",Yugo & Lala,神秘世界历险记,Movie,1,"Aug 10, 2012",Unknown,Unknown,Unknown,Unknown,Original,1 hr. 27 min.,PG - Children,16096,17953,205,0,17,50,5,63,70,6,4,1,2,5,2,1,7,Unknown,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49827,Kidou Senshi Gundam: Cucuruz Doan no Shima,,"Action, Mecha",Mobile Suit Gundam:Cucuruz Doan's Island,機動戦士ガンダム ククルス・ドアンの島,Movie,1,"Jun 3, 2022",Unknown,Bandai Namco Filmworks,Unknown,Sunrise,Other,1 hr. 48 min.,PG-13 - Teens 13 or older,2308,6936,6547,23,202,2775,68,52,3450,104,261,752,729,275,93,23,10,8,9,Action\n \n\n Action,Mecha\n \n\n Mecha,NaN,NaN,NaN,NaN,NaN


In [31]:
df.to_csv(f"{BASE_PATH}/anime.tsv", index=False, sep="\t", encoding="UTF-8")


# 6. Create rating_complete.csv

This file only contain animes with `watching_status==2`(complete) and have been rated (`score!=0`).

In [32]:
if not os.path.exists(f"{BASE_PATH}/rating_complete.csv"):
    with open(f"{BASE_PATH}/rating_complete.csv", "w", encoding="UTF-8") as file:
        file.write("user_id,anime_id,rating\n")

In [33]:
unique_anime = set()
all_users = sorted(os.listdir(USER_PATH), key=lambda x:int(x.split(".")[0]))

with open(f"{BASE_PATH}/rating_complete.csv", "a") as f1:

    for i, user_file in enumerate(all_users):
        if not user_file.endswith(".csv"):
            continue

        print(f"\r{i+1}/{len(all_users)}", end="")

        user_id = user_file.split(".")[0]
        with open(f"{USER_PATH}/{user_file}", "r") as file:
            file.readline()
            for line in file:
                anime_id, score, watching_status, _ = line.strip().split(",")
                if int(watching_status) == 2 and (score) != 0:
                    f1.write(f"{user_id},{anime_id},{score}\n")

577/577

# 7. Unified animelist

In [34]:
if not os.path.exists(f"{BASE_PATH}/animelist.csv"):
    with open(f"{BASE_PATH}/animelist.csv", "w", encoding="UTF-8") as file:
        file.write("user_id,anime_id,rating,watching_status,watched_episodes\n")

In [35]:
unique_anime = set()
all_users = sorted(os.listdir(USER_PATH), key=lambda x:int(x.split(".")[0]))

with open(f"{BASE_PATH}/animelist.csv", "a") as f1:

    for i, user_file in enumerate(all_users):
        if not user_file.endswith(".csv"):
            continue

        print(f"\r{i+1}/{len(all_users)}", end="")

        user_id = user_file.split(".")[0]
        with open(f"{USER_PATH}/{user_file}", "r") as file:
            file.readline()
            for line in file:
                anime_id, score, watching_status, watched_episodes = line.strip().split(",")
                f1.write(f"{user_id},{anime_id},{score},{watching_status},{watched_episodes}\n")

577/577